In [1]:
import sys
!{sys.executable} -m pip install antspyx

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 7.7 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for antspyx (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [27 lines of output]
      *** scikit-build-core 0.11.5 using CMake 4.0.0 (wheel)
      *** Configuring CMake...
      loading initial cache file build/cp39-cp39-macosx_14_0_arm64/CMakeInit.txt
      -- The CXX compiler identification is AppleClang 15.0.0.15000309
      -- Detecting CXX compiler ABI info
      -- Detecting CXX compiler ABI info - done
      -- Check for working CXX compiler: /usr/bin/clang++ - skipped
      -- Detecting CXX compile features
      -- Detecting CXX compile features - done
      -- U

In [2]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn import preprocessing
import ants


def normalize_slice(slice_data):
    # Clip data to 0.5–99.5 percentile
    lower, upper = np.percentile(slice_data, [0.5, 99.5])
    slice_clipped = np.clip(slice_data, lower, upper)
    # Normalize all values to be between [0, 1]
    normalized = (slice_clipped - lower) / (
        upper - lower + 1e-8
    )  # prevent divion by zero
    return normalized


def visualize(subject):
    # Load the images
    followup = nib.load(
        f"../ABIDE/ABIDE_II_BIDS/derivatives/MNI/{subject}/anat/{subject}_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz"
    )
    followup_mask = nib.load(
        f"../ABIDE/ABIDE_II_BIDS/derivatives/MNI/{subject}/anat/{subject}_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz"
    )
    baseline = nib.load(
        f"../ABIDE/ABIDE_II_BIDS_Baseline/derivatives/MNI/{subject}/anat/{subject}_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz"
    )
    baseline_mask = nib.load(
        f"../ABIDE/ABIDE_II_BIDS_Baseline/derivatives/MNI/{subject}/anat/{subject}_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz"
    )

    # Get the image data as np arrays
    baseline_data = baseline.get_fdata()
    followup_data = followup.get_fdata()
    followup_mask_data = followup_mask.get_fdata()
    baseline_mask_data = baseline_mask.get_fdata()

    # Apply the mask: set non-brain voxels to 0
    masked_followup = followup_data * followup_mask_data
    masked_baseline = baseline_data * baseline_mask_data

    # Extract middle slices
    followup_slice_x = masked_followup[
        masked_followup.shape[0] // 2, :, :
    ]  # slice along X-axis
    followup_slice_y = masked_followup[
        :, masked_followup.shape[1] // 2, :
    ]  # slice along Y-axis
    followup_slice_z = masked_followup[
        :, :, masked_followup.shape[2] // 2
    ]  # slice along Z-axis
    baseline_slice_x = masked_baseline[
        masked_baseline.shape[0] // 2, :, :
    ]  # slice along X-axis
    baseline_slice_y = masked_baseline[
        :, masked_baseline.shape[1] // 2, :
    ]  # slice along Y-axis
    baseline_slice_z = masked_baseline[
        :, :, masked_baseline.shape[2] // 2
    ]  # slice along Z-axis

    # Normalize slices before plotting
    baseline_slice_x = normalize_slice(baseline_slice_x)
    baseline_slice_y = normalize_slice(baseline_slice_y)
    baseline_slice_z = normalize_slice(baseline_slice_z)
    followup_slice_x = normalize_slice(followup_slice_x)
    followup_slice_y = normalize_slice(followup_slice_y)
    followup_slice_z = normalize_slice(followup_slice_z)

    # Get Jacobian determinant map comparing baseline vs. followup
    baseline_ants = ants.from_numpy(baseline_data)
    followup_ants = ants.from_numpy(followup_data)
    reg = ants.registration(
        fixed=baseline_ants, moving=followup_ants, type_of_transform="SyN"
    )
    jacobian = ants.create_jacobian_determinant_image(
        domain_image=baseline_ants, tx=reg["fwdtransforms"][0], do_log=False
    )
    jac_data = jacobian.numpy()

    # Plot all three slices side by side in each of 3 dimensions
    fig, axes = plt.subplots(3, 3, figsize=(15, 6))
    # Baseline slices
    axes[0, 0].imshow(baseline_slice_x.T, cmap="gray", origin="lower")
    axes[0, 0].set_title("Baseline: Middle slice X-axis")
    axes[0, 0].axis("off")

    axes[0, 1].imshow(baseline_slice_y.T, cmap="gray", origin="lower")
    axes[0, 1].set_title("Baseline: Middle slice Y-axis")
    axes[0, 1].axis("off")

    axes[0, 2].imshow(baseline_slice_z.T, cmap="gray", origin="lower")
    axes[0, 2].set_title("Baseline: Middle slice Z-axis")
    axes[0, 2].axis("off")

    # Row 2: Followup slices
    axes[1, 0].imshow(followup_slice_x.T, cmap="gray", origin="lower")
    axes[1, 0].set_title("Followup: Middle slice X-axis")
    axes[1, 0].axis("off")

    axes[1, 1].imshow(followup_slice_y.T, cmap="gray", origin="lower")
    axes[1, 1].set_title("Followup: Middle slice Y-axis")
    axes[1, 1].axis("off")

    axes[1, 2].imshow(followup_slice_z.T, cmap="gray", origin="lower")
    axes[1, 2].set_title("Followup: Middle slice Z-axis")
    axes[1, 2].axis("off")

    # Row 3: Jacobian map slices
    im_x = axes[2, 0].imshow(
        jac_data[jac_data.shape[0] // 2, :, :].T, cmap="bwr", origin="lower"
    )
    axes[2, 0].set_title("Jacobian: Middle slice X-axis")
    axes[2, 0].axis("off")
    fig.colorbar(
        im_x, ax=axes[2, 0], fraction=0.046, pad=0.04, label="Jacobian Determinant"
    )

    im_y = axes[2, 1].imshow(
        jac_data[:, jac_data.shape[1] // 2, :].T, cmap="bwr", origin="lower"
    )
    axes[2, 1].set_title("Jacobian: Middle slice Y-axis")
    axes[2, 1].axis("off")
    fig.colorbar(
        im_y, ax=axes[2, 1], fraction=0.046, pad=0.04, label="Jacobian Determinant"
    )

    im_z = axes[2, 2].imshow(
        jac_data[:, :, jac_data.shape[2] // 2].T, cmap="bwr", origin="lower"
    )
    axes[2, 2].set_title("Jacobian: Middle slice Z-axis")
    axes[2, 2].axis("off")
    fig.colorbar(
        im_z, ax=axes[2, 2], fraction=0.046, pad=0.04, label="Jacobian Determinant"
    )

    plt.show()
    return jac_data


def get_ages(subject):
    df = pd.read_csv("../ABIDE/ABIDEII_Long_Composite_Phenotypic.csv")

    # Find baseline and followup ages for a specific subject
    sub_id = int(subject[4:])
    baseline = df[(df["SUB_ID"] == sub_id) & (df["SESSION"] == "Baseline")][
        "AGE_AT_SCAN "
    ].values[0]
    followup = df[(df["SUB_ID"] == sub_id) & (df["SESSION"] == "Followup_1")][
        "AGE_AT_SCAN "
    ].values[0]

    # Print the results
    print(f"Subject {sub_id} Baseline Age: {baseline}")
    print(f"Subject {sub_id} Followup Age: {followup}")
    return baseline, followup

    """
    ---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[2], line 135
    132         continue 
    134 if jacobian_list: # check it's not empty
--> 135     jacobian_stack = np.stack(jacobian_list, axis=0)  # shape: (N, X, Y, Z)
    136     jacobian_mean = np.mean(jacobian_stack, axis=0)
    138     # Visualize the average Jacobian map

File ~/.local/lib/python3.10/site-packages/numpy/core/shape_base.py:449, in stack(arrays, axis, out, dtype, casting)
    447 shapes = {arr.shape for arr in arrays}
    448 if len(shapes) != 1:
--> 449     raise ValueError('all input arrays must have the same shape')
    451 result_ndim = arrays[0].ndim + 1
    452 axis = normalize_axis_index(axis, result_ndim)

ValueError: all input arrays must have the same shape"""

ModuleNotFoundError: No module named 'ants'

In [ ]:
from utils.utils import get_ABIDE_II_subject_followup
from utils.transforms import get_train_data_transform

import monai
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch

In [ ]:
samples = get_ABIDE_II_subject_followup()
train_transform = get_train_data_transform()
# val_transform = get_val_data_augmentation_transform()
train_ds = Dataset(data=samples, transform=train_transform)
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=8)

In [ ]:
for step, train_batch_data in enumerate(train_loader):
    # Get the subject ID from the batch data
    subject = train_batch_data["subject_id"][0]
    print(f"Processing subject: {subject}")

    # Visualize the subject's MRI data
    jacobian_map = visualize(subject)

    # Get ages for the subject
    baseline_age, followup_age = get_ages(subject)

    # Optionally, you can save or process the jacobian_map further here
    # For now, we just print the shape of the jacobian map
    print(f"Jacobian map shape for {subject}: {jacobian_map.shape}")

In [ ]:
# Visualize for each subject that has baseline and followup
jacobian_list = []
subdirectories = [
    entry.name
    for entry in os.scandir("../ABIDE/ABIDE_II_BIDS_Baseline/derivatives/MNI")
    if entry.is_dir()
]
for subject in subdirectories:
    if subject == "logs":  # additional folder in directory that is not a subject - skip
        continue
    try:
        baseline, followup = get_ages(subject)
        jac_data = visualize(subject)
        jacobian_list.append(jac_data)
    except Exception as e:
        print(f"Skipping {subject} due to error: {e}")
        continue

if jacobian_list:  # check it's not empty
    jacobian_stack = np.stack(jacobian_list, axis=0)  # shape: (N, X, Y, Z)
    jacobian_mean = np.mean(jacobian_stack, axis=0)

    # Visualize the average Jacobian map
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    axes[0].imshow(
        jacobian_mean[jacobian_mean.shape[0] // 2, :, :].T, cmap="bwr", origin="lower"
    )
    axes[0].set_title("Mean Jacobian: X slice")
    axes[0].axis("off")
    axes[1].imshow(
        jacobian_mean[:, jacobian_mean.shape[1] // 2, :].T, cmap="bwr", origin="lower"
    )
    axes[1].set_title("Mean Jacobian: Y slice")
    axes[1].axis("off")
    axes[2].imshow(
        jacobian_mean[:, :, jacobian_mean.shape[2] // 2].T, cmap="bwr", origin="lower"
    )
    axes[2].set_title("Mean Jacobian: Z slice")
    axes[2].axis("off")
    plt.suptitle("Average Jacobian Determinant Map")
    plt.show()